In [1]:
%load_ext autoreload
%autoreload 2
import tensorflow_quantum as tfq
from utilities.variational import VQE
from utilities.qmodels import *
from utilities.evaluator import Evaluator
from utilities.idinserter import IdInserter
from utilities.unitary_killer import UnitaryMurder
from utilities.simplifier import Simplifier
import cirq
import numpy as np

In [31]:

vqe_handler = VQE(n_qubits=2, problem_config={"problem":"XXZ","g":1,"J":2})
iid_inserter = IdInserter(n_qubits = vqe_handler.n_qubits, selector_temperature=40, epsilon=0.1)
simplifier = Simplifier(n_qubits=vqe_handler.n_qubits,testing=True)

#indexed_circuit = [vqe_handler.number_of_cnots + k for k in range(vqe_handler.n_qubits, 2*vqe_handler.n_qubits )]
#indexed_circuit = [5,4,2,4,3,5,3,0,5,5]
#indexed_circuit = [5,3,3,5,4,0,5,5,3,3,5,3,5]
indexed_circuit =[5, 3, 5, 3, 4, 2, 4, 1, 3, 1]
circuit, symbols, index_to_symbols = vqe_handler.give_circuit(indexed_circuit)
energy, resolver, history = vqe_handler.vqe(indexed_circuit)

model = QNN(symbols=list(resolver.keys()), observable=vqe_handler.observable)
print(model(cirq.resolve_parameters(circuit,resolver)), energy)


tf.Tensor([-3.9999928], shape=(1,), dtype=float32) tf.Tensor(-3.9999938, shape=(), dtype=float32)


In [45]:
resolver["th_7"] = 20

In [46]:
i,r,po=simplifier.simplify_step(indexed_circuit,resolver,index_to_symbols)

5
cnt_rule 5


In [47]:
cirq.resolve_parameters(circuit,resolver)

(0, 0): ───Rx(1.49π)─────Rz(π)────────Rx(0.487π)────────────────X────────────────X───
                                                                │                │
(0, 1): ───Rx(-0.998π)───Rz(0.124π)───Rx(0.998π)───Rz(-1.05π)───@───Rz(-1.63π)───@───

In [48]:
nu=vqe_handler.give_unitary(i,r)
nu

(0, 0): ───Rx(1.49π)────Rz(π)───────Rx(0.487π)───X────────────────X───
                                                 │                │
(0, 1): ───Rz(-1.56π)───Rx(-2.0π)───Rz(-1.61π)───@───Rz(0.124π)───@───

In [52]:
vqe_handler.give_circuit(i)[0]

(0, 0): ───Rx(th_3)───Rz(th_4)───Rx(th_5)───X──────────────X───
                                            │              │
(0, 1): ───Rz(th_0)───Rx(th_1)───Rz(th_2)───@───Rz(th_6)───@───

In [54]:
{s:v/np.pi for s,v in r.items()}

{'th_0': 2.438488638105051,
 'th_1': 2.000727394910686,
 'th_2': -1.6112762314978026,
 'th_3': 1.4863957847889575,
 'th_4': 1.0002777129627916,
 'th_5': -3.5134878616778744,
 'th_6': 0.12442375369909131}

In [40]:
q=vqe_handler.qubits[0]
cirq.unitary(cirq.Circuit([cirq.rx(-0.998*np.pi).on(q),cirq.rz(.124*np.pi).on(q),cirq.rx(.998*np.pi).on(q),cirq.rz(-1.05*np.pi).on(q)]))

array([[-2.69924486e-01+0.96288073j, -9.54140414e-05+0.00121235j],
       [ 9.54140414e-05+0.00121235j, -2.69924486e-01-0.96288073j]])

In [41]:
cirq.unitary(cirq.Circuit([cirq.rz(1.44*np.pi).on(q),cirq.rx(2*np.pi).on(q),cirq.rz(-0.611*np.pi).on(q)]))

array([[-0.26538785+0.96414174j,  0.        +0.j        ],
       [-0.        +0.j        , -0.26538785-0.96414174j]])

In [43]:
model(nu)

<tf.Tensor: shape=(1,), dtype=float32, numpy=array([-3.9999866], dtype=float32)>

In [182]:

vqe_handler = VQE(n_qubits=2, problem_config={"problem":"XXZ","g":1,"J":2})
iid_inserter = IdInserter(n_qubits = vqe_handler.n_qubits, selector_temperature=40, epsilon=0.1)
simplifier = Simplifier(n_qubits=vqe_handler.n_qubits,testing=True)

#indexed_circuit = [vqe_handler.number_of_cnots + k for k in range(vqe_handler.n_qubits, 2*vqe_handler.n_qubits )]
indexed_circuit = [5,5,5,1,5,5,5]
#indexed_circuit = [5,3,3,5,4,0,5,5,3,3,5,3,5]
circuit, symbols, index_to_symbols = vqe_handler.give_circuit(indexed_circuit)
resolver = {s:r for s,r in zip(symbols, np.random.randn(len(symbols)))}
#energy, resolver, history = vqe_handler.vqe(indexed_circuit)

model = QNN(symbols=list(resolver.keys()), observable=vqe_handler.observable)
print(model(cirq.resolve_parameters(circuit,resolver)))


tf.Tensor([-1.035732], shape=(1,), dtype=float32)


In [183]:
i,r,po=simplifier.simplify_step(indexed_circuit,resolver,index_to_symbols)

4
here:  ['th_1', 'th_2']
0.8779985591597353 th_0
4
here:  ['th_4', 'th_5']
1.9401020044321542 th_1


In [184]:
nu=vqe_handler.give_unitary(i,r)
nu
model(nu)

<tf.Tensor: shape=(1,), dtype=float32, numpy=array([-1.0357327], dtype=float32)>